In [80]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from sklearn.linear_model import Lasso
from sklearn.metrics import roc_auc_score, roc_curve
%matplotlib inline

In [2]:
raw_df = pd.read_csv('rosbank_train.csv')
raw_df.head(5)

,PERIOD,cl_id,MCC,channel_type,currency,TRDATETIME,amount,trx_category,target_flag,target_sum
0,01/10/2017,0,5200,NaN,810,21OCT17:00:00:00,5023.0,POS,0,0.0
1,01/10/2017,0,6011,NaN,810,12OCT17:12:24:07,20000.0,DEPOSIT,0,0.0
2,01/12/2017,0,5921,NaN,810,05DEC17:00:00:00,767.0,POS,0,0.0
3,01/10/2017,0,5411,NaN,810,21OCT17:00:00:00,2031.0,POS,0,0.0
4,01/10/2017,0,6012,NaN,810,24OCT17:13:14:24,36562.0,C2C_OUT,0,0.0


In [3]:
len(raw_df)

490513

In [4]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 490513 entries, 0 to 490512
Data columns (total 10 columns):
PERIOD          490513 non-null object
cl_id           490513 non-null int64
MCC             490513 non-null int64
channel_type    487603 non-null object
currency        490513 non-null int64
TRDATETIME      490513 non-null object
amount          490513 non-null float64
trx_category    490513 non-null object
target_flag     490513 non-null int64
target_sum      490513 non-null float64
dtypes: float64(2), int64(4), object(4)
memory usage: 37.4+ MB


In [5]:
len(raw_df.cl_id.unique())

5000

In [6]:
cl_ids_test = np.random.choice(raw_df.cl_id.unique(), size=1000, replace=False)
cl_ids_test_set = set(cl_ids_test)

In [30]:
transaction_train = raw_df[~raw_df.cl_id.isin(cl_ids_test)].copy()
transaction_test = raw_df[raw_df.cl_id.isin(cl_ids_test)].copy()

Наборы данных вида Transactions (несколько транзакций на одного клиента) трансформировать в таблицу, где cl_id будут уникальными (соответственно 4000 строк в train и 1000 строк в test

In [44]:
def uniqalize(dataset):
    return dataset[['cl_id', 'target_flag']].drop_duplicates();

def sum_amount(rich_dataset):
    return pd.DataFrame(rich_dataset.groupby('cl_id')['amount'].sum()).reset_index()

def sum_amount_by_currency(rich_dataset, currency):
    return pd.DataFrame(rich_dataset[rich_dataset['currency'] == currency]
                        .groupby('cl_id')['amount'].sum()).reset_index().rename(columns={'amount': currency})

def transaction_by_day_count(dataset):
    dataset_copy = dataset.copy()
    dataset_copy['day_of_transaction'] = dataset_copy['TRDATETIME'].apply( lambda x: (datetime.datetime.strptime(x, '%d%b%y:%H:%M:%S')).strftime('%w') ) 
    return dataset_copy.groupby(['cl_id','day_of_transaction'])['day_of_transaction'].size().unstack().fillna(0).add_prefix('count_day_').reset_index()

def transaction_by_hour_count(dataset):
    dataset_copy = dataset.copy()
    dataset_copy['hour_of_transaction'] = dataset_copy['TRDATETIME'].apply( lambda x: (datetime.datetime.strptime(x, '%d%b%y:%H:%M:%S')).strftime('%H') ) 
    return dataset_copy.groupby(['cl_id','hour_of_transaction'])['hour_of_transaction'].size().unstack().fillna(0).add_prefix('count_hour_').reset_index()

def transaction_by_day_sum(dataset):
    return dataset.groupby(['cl_id', 'day_of_transaction'])['amount'].sum().unstack().fillna(0).reset_index().add_prefix('sum_day_')

def transaction_by_hour_sum(dataset):
    return dataset.groupby(['cl_id', 'hour_of_transaction'])['amount'].sum().unstack().fillna(0).reset_index().add_prefix('sum_hour_')

In [40]:
from sklearn.linear_model import LogisticRegression

In [56]:
def make_features(dataset):
    uniq_dataset = dataset.copy()
    uniq_dataset = uniqalize(dataset)
    uniq_dataset = pd.merge(uniq_dataset,sum_amount(dataset))
    uniq_dataset = pd.merge(uniq_dataset,sum_amount_by_currency(dataset, 810),  how='outer')
    uniq_dataset = pd.merge(uniq_dataset,sum_amount_by_currency(dataset, 978), how='outer')
    uniq_dataset = pd.merge(uniq_dataset,sum_amount_by_currency(dataset, 840), how='outer')
    uniq_dataset = pd.merge(uniq_dataset,transaction_by_day_count(dataset), how='outer')
    uniq_dataset = pd.merge(uniq_dataset,transaction_by_hour_count(dataset), how='outer')
    return uniq_dataset.fillna(0)

In [57]:
def get_features(dataset):
    return dataset.loc[:, dataset.columns != 'target_flag']

In [58]:
x = get_features(make_features(transaction_train))
y = make_features(transaction_train)['target_flag']
x_test = get_features(make_features(transaction_test))
y_test = make_features(transaction_test)['target_flag']
model = LogisticRegression()
model.fit(x, y)
model.score(x_test, y_test)

0.691

In [71]:
rg = Lasso(alpha=0.01).fit(x, y)

In [75]:
rg.coef_

array([-5.98993883e-05,  8.91578920e-09,  1.92531332e-08,  1.89286800e-05,
        4.04904712e-06,  7.35746371e-04, -2.05194191e-04,  8.84945814e-04,
        2.77381273e-03, -0.00000000e+00,  4.55471395e-04,  9.00094497e-04,
        4.80533436e-04,  0.00000000e+00,  0.00000000e+00,  2.79026852e-03,
        0.00000000e+00,  1.15214632e-02, -0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00, -2.22493427e-03, -0.00000000e+00,
       -4.58303788e-03,  1.89436707e-03,  0.00000000e+00,  0.00000000e+00,
       -0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00,  0.00000000e+00,  0.00000000e+00])

In [81]:
y_train_predict = model.predict_proba(x)[:, 1]
y_test_predict = model.predict_proba(x_test)[:, 1]
roc_auc_train = np.round(roc_auc_score(y, y_train_predict), 2)
roc_auc_test = np.round(roc_auc_score(y_test, y_test_predict), 2)
print("Train: ", roc_auc_train)
print("Test: ", roc_auc_test)

Train:  0.75
Test:  0.75


In [104]:
transaction_train.groupby('MCC').size().sort_values(ascending=False)

MCC
5411    95600
6011    43093
5814    32540
5812    23586
5499    21263
5541    15883
5912    14886
5999    10431
6012     7936
5921     6515
5331     5909
4829     4973
5211     4967
4121     4890
5691     4060
5261     3774
4111     3406
5977     3392
5200     2699
5732     2599
5311     2597
5533     2316
5651     2287
5641     2207
4812     2180
8999     2080
8099     2067
5661     2014
5722     2003
5941     1989
        ...  
7012        1
5998        1
3628        1
5937        1
3032        1
8675        1
3625        1
7519        1
3515        1
3513        1
3535        1
3508        1
8244        1
3543        1
3387        1
3583        1
5139        1
3659        1
3236        1
3778        1
3191        1
3730        1
3181        1
3715        1
7631        1
3520        1
3607        1
3057        1
3051        1
5960        1
Length: 333, dtype: int64